In [4]:
import psi4
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv


psi4.set_options({'reference' : 'uhf',
                  'guess' : 'gwh',
                  'MAXITER'   :  150})


## Generates the energies for different diatomics. Atom_list provides all of the diatomics from: https://cccbdb.nist.gov/expdiatomicsx.asp

In [6]:
#atom = ['H', 'Li', 'Li', 'Be', 'B', 'B', 'C', 'C', 'N', 'B',  'C', 'N', 'O', 'Li', 'Be', 'B', 'C', 'N', 'O', 'H', 'Li',  'B', 'C', 'N', 'F', 'F', 'Na', 'Na', 'Na', 'Na', 'Mg', 'Mg', 'Mg', 'Al', 'Al', 'Al', 'Al', 'Al', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'P', 'C', 'P', 'P', 'P', 'Si', 'P',  'H', 'Li', 'Be', 'B', 'C', 'N', 'S', 'S', 'Na', 'Mg', 'Al', 'Si', 'S', 'H',   'Li', 'B', 'C'  ,'N' , 'Cl', 'Cl', 'Na',   'Al', 'Si', 'P', 'Cl' ]  

atom_list1 = ['H', 'Li', 'Li', 'Be', 'B', 'B', 'C', 'N', 'B',  'C', 'N', 'O', 'Li', 'Be', 'B', 'C', 'N', 'O', 'H', 'Li',  'B', 'C', 'N', 'F', 'F', 'Na', 'Na', 'Na', 'Na', 'Mg', 'Mg', 'Mg', 'Al', 'Al', 'Al', 'Al', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'P', 'P', 'P', 'P', 'Si', 'P',  'H', 'Li', 'Be', 'B', 'C', 'N', 'S', 'S', 'Na', 'Mg', 'Al', 'Si', 'S', 'H',   'Li', 'B', 'C'  ,'N' , 'Cl', 'Cl', 'Na',   'Al', 'Si', 'P', 'Cl' ]  
atom_list2 = ['H', 'H',  'Li', 'H', 'H', 'B',  'C', 'H', 'N', 'N',  'N', 'H', 'O',  'O', 'O',  'O', 'O', 'O', 'F',  'F',  'F', 'F', 'F', 'O', 'F', 'H', 'Li',  'F',  'Na', 'H', 'O',  'Mg' , 'H',  'C',  'F', 'Al' , 'H', 'C',   'N' , 'O' ,  'F', 'Si', 'H', 'N', 'O', 'F' , 'P', 'P',  'S', 'S',  'S',  'S', 'S', 'S', 'O',  'F',  'S', 'S', 'S' , 'S',  'S',  'Cl', 'Cl', 'Cl', 'Cl', 'Cl', 'O',  'F',  'Cl',  'Cl', 'Cl',  'Cl', 'Cl']


#atoms_list_master = ['H', 'Li','Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br']
#atoms_list_z =     [ 1,     3,   4,   5,   6,  7,     8,   9,   11,  12,   13,   14,   15,  16,  17,   19,   20,  29,  30,    31,   32,   33,   34,   35 ]

#Molecules complaining
#CH(6), AlO(35), CP(45)

atoms={"H":1,"He":2, "Li":3, "Be":4, "B":5, "C":6, "N":7, "O":8, "F":9,
         "Ne":10, "Na":11, "Mg":12, "Al":13, "Si":14, "P":15, "S":16, 
         "Cl":17, "Ar":18, "K":19, "Ca":20, "Sc":21, "Ti":22, "V":23,
         "Cr":24, "Mn":25, "Fe":26, "Co":27, "Ni":28, "Cu":29, "Zn":30,
         "Ga":31, "Ge":32, "As":33, "Se":34, "Br":35, "Kr":36} 

assert len(atom_list1) == len(atom_list2)

In [ ]:
psi4.core.clean()   


diatomic_even_m = """
0 1

**atom1** 0.0 0.0 **R**
**atom2** 0.0 0.0 0.0 

symmetry c1
units bohr
"""

diatomic_odd_m = """
0 2

**atom1** 0.0 0.0 **R**
**atom2** 0.0 0.0 0.0 

symmetry c1
units bohr

"""

diatomic_energies = {'atom1':[], 'atom2':[], 'r':[], 'hf':[], 'dft':[], 'ccsdt':[], 'difference':[]}
#diatomic_energies = {'atom1':[], 'atom2':[], 'r':[], 'hf':[], 'ccsdt':[], 'difference':[]}


distances = np.linspace(0.4, 7, 60)

index = 0
for r in distances:

    atom1 = atom_list1[index]
    atom2 = atom_list2[index]

    i = atoms[atom_list1[index]]
    j = atoms[atom_list2[index]]

    if (i+j) % 2 == 0: 

        print(F'Now calculating {atom1}{atom2} at distance {r} with {i},{j} = {i+j} -> even')
        mol=psi4.geometry(diatomic_even_m.replace('**atom1**', atom1).replace('**atom2**', atom2).replace('**R**', str(r)))
        en_ccsdt = psi4.energy('ccsd(t)/6-311G(d)', molecule=mol, units='Hartree')
        en_hf = psi4.energy('hf/6-311G(d)', molecule=mol, units='Hartree')
        en_ccsdt = psi4.energy('ccsd(t)/cc-pvdz', molecule=mol, units='Hartree')
        en_hf = psi4.energy('hf/cc-pvdz', molecule=mol, units='Hartree')
        #en_dft = psi4.energy('svwn/6-311G(d)', molecule=mol, units='Hartree', reference='uks')
        diatomic_energies["atom1"].append(atom1)
        diatomic_energies["atom2"].append(atom2)
        diatomic_energies["r"].append(r)
        diatomic_energies["hf"].append(en_hf)
        #diatomic_energies["dft"].append(en_dft)
        diatomic_energies["ccsdt"].append(en_ccsdt)        
        diatomic_energies["difference"].append(en_hf-en_ccsdt)  



    if (i+j) % 2 != 0:

        print(F'Now calculating {atom1}{atom2} at distance {r} with {i},{j} = {i+j} -> odd')
        mol=psi4.geometry(diatomic_odd_m.replace('**atom1**', atom1).replace('**atom2**', atom2).replace('**R**', str(r)))
        en_ccsdt = psi4.energy('ccsd(t)/6-311G(d)', molecule=mol, units='Hartree')
        en_hf = psi4.energy('hf/6-311G(d)', molecule=mol, units='Hartree')
        #en_dft = psi4.energy('svwn/6-311G(d)', molecule=mol, units='Hartree', reference='uks')
        diatomic_energies["atom1"].append(atom1)
        diatomic_energies["atom2"].append(atom2)
        diatomic_energies["r"].append(r)
        diatomic_energies["hf"].append(en_hf)
        #diatomic_energies["dft"].append(en_dft)
        diatomic_energies["ccsdt"].append(en_ccsdt)
        diatomic_energies["difference"].append(en_hf-en_ccsdt)  


psi4.core.clean() 

In [ ]:
#Print dataframe with all the diatomic energy information
pd.set_option('display.max_rows', None) 
diatomics_pd = pd.DataFrame(diatomic_energies)
diatomics_pd